# SQL query from table names - Continued

In [13]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
load_dotenv("OpenAI.env")
OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [16]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [19]:
context = [ {'role':'system', 'content':"""
 CREATE SEVERAL (3+) TABLES HERE
"""} ]



In [21]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
"""
})

In [23]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [26]:
#new
context_user = context.copy()
print(return_CCRMSQL("""YOUR QUERY HERE""", context_user))

I see that you have not provided any tables for me to work with. Could you please provide the tables so that I can assist you with your queries?


In [28]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("YOUR QUERY HERE", old_context_user))

This is your SQL:
```sql

SELECT * 
FROM employees;
```

This SQL query selects all data from the "employees" table.


In [30]:
#new
print(return_CCRMSQL("YOUR QUERY HERE", context_user))

I see you have not provided any tables for me to work with. Could you please provide the tables so that I can assist you with your queries?


In [32]:
#old
print(return_CCRMSQL("YOUR QUERY HERE", old_context_user))

This is your SQL:
```sql

SELECT * 
FROM employees;
```
This SQL query selects all data from the "employees" table.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [35]:
context = [{'role': 'system', 'content': """
-- Create the tables with sample data for employees, salary, and studies.
CREATE TABLE employees (
    ID_usr INT PRIMARY KEY,
    name VARCHAR(100)
);

INSERT INTO employees (ID_usr, name) VALUES
(1, 'Alice'),
(2, 'Bob'),
(3, 'Charlie');

CREATE TABLE salary (
    ID_usr INT PRIMARY KEY,
    year DATE,
    salary FLOAT,
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
);

INSERT INTO salary (ID_usr, year, salary) VALUES
(1, '2025-01-01', 80000.00),
(2, '2025-01-01', 95000.00),
(3, '2025-01-01', 70000.00);

CREATE TABLE studies (
    ID INT PRIMARY KEY,
    ID_usr INT,
    educational_level INT,
    Institution VARCHAR(100),
    Years DATE,
    Speciality VARCHAR(100),
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
);

INSERT INTO studies (ID, ID_usr, educational_level, Institution, Years, Speciality) VALUES
(1, 1, 4, 'MIT', '2015-05-01', 'Computer Science'),
(2, 2, 5, 'Harvard', '2016-06-01', 'Economics'),
(3, 3, 3, 'Stanford', '2017-07-01', 'Biology');
""" 
}]

context.append({'role': 'system', 'content': """
-- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.

-- Sample 1
-- Query to get the name of the most educated employee.
SELECT e.name
FROM employees e
JOIN studies st ON e.ID_usr = st.ID_usr
ORDER BY st.educational_level DESC
LIMIT 1;
"""
})

# User Query: Find the name of the employee with the highest educational level
user_message = "Who has the highest level of education?"
context_user = context.copy()
print(return_CCRMSQL(user_message, context_user))


Alice


In [39]:


# User Query: Get the average salary by institution
user_message = "What is the average salary for each institution?"
context_user = context.copy()
print(return_CCRMSQL(user_message, context_user))


Here is the query to get the average salary for each institution:

```sql
SELECT st.Institution, AVG(sa.salary) AS avg_salary
FROM studies st
JOIN employees e ON st.ID_usr = e.ID_usr
JOIN salary sa ON e.ID_usr = sa.ID_usr
GROUP BY st.Institution
ORDER BY avg_salary DESC;
```

This query will provide the average salary for each institution, sorted in descending order of average salary.


In [41]:


# User Query: Get the total salary expense for each institution
user_message = "What is the total salary expense by institution?"
context_user = context.copy()
print(return_CCRMSQL(user_message, context_user))


```sql
-- Query to get the total salary expense by institution.
SELECT st.Institution, SUM(sa.salary) AS total_salary_expense
FROM studies st
JOIN employees e ON st.ID_Usr = e.ID_Usr
JOIN salary sa ON e.ID_Usr = sa.ID_Usr
GROUP BY st.Institution
ORDER BY total_salary_expense DESC;
```
